# Kepler's Laws of Planetary Motion

This notebook contains the programmatic verification for the **Kepler's Laws of Planetary Motion** entry from the THEORIA dataset.

**Entry ID:** keplers_laws  
**Required Library:** sympy 1.12.0

## Description
Kepler's three laws describe planetary motion around the Sun: orbits are ellipses with the Sun at one focus, planets sweep equal areas in equal times, and the square of orbital period is proportional to the cube of the semi-major axis. These empirical laws provided the foundation for Newton's law of gravitation and remain fundamental to celestial mechanics and orbital dynamics.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.12.0

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# Define symbols
r, a, e, theta, L, m, t, T, G, M, u, E, h, p, b = sp.symbols('r a e theta L m t T G M u E h p b', positive=True, real=True)
pi = sp.pi

# Steps 1-2: Newton's law of gravitation and torque calculation
F_grav = G*M*m/r**2
# Central force means tau = r x F = 0 (parallel vectors)

# Steps 3-4: Angular momentum conservation
# From dot(L) = tau = 0, we get L = constant
L_magnitude = m*r**2*sp.Symbol('omega')

# Steps 5-6: Angular momentum and areal velocity
# L = m*r²*dot(theta) and dot(A) = (1/2)*r²*dot(theta) = L/(2*m)
omega_from_L = L/(m*r**2)
areal_velocity = L/(2*m)
dot_A_direct = sp.Rational(1,2)*r**2*omega_from_L
assert sp.simplify(dot_A_direct - areal_velocity) == 0

# Steps 7-11: Polar coordinates and specific angular momentum
# From central force: d/dt(r²*dot(theta)) = 0
# So h := r²*dot(theta) = constant (specific angular momentum)
h_def = r**2*omega_from_L
h_simplified = L/m
assert sp.simplify(h_def - h_simplified) == 0

# Steps 12-14: Radial equation of motion
# ddot(r) - h²/r³ = -GM/r²

# Step 15: Binet substitution u = 1/r
u_def = 1/r
r_from_u = 1/u
assert sp.simplify(r_from_u.subs(u, u_def) - r) == 0

# Steps 16-18: Transform to orbital differential equation
# Final form: d²u/dθ² + u = GM/h²
orbital_eq_rhs = G*M/h**2

# Steps 19-20: Solution of orbital equation
# u(theta) = (GM/h²)*(1 + e*cos(theta))
u_solution = orbital_eq_rhs*(1 + e*sp.cos(theta))

# Steps 21-22: Convert back to r(theta) and define semi-latus rectum
# r(theta) = h²/(GM*(1 + e*cos(theta)))
# p := h²/(GM), so r(theta) = p/(1 + e*cos(theta))
p_def = h**2/(G*M)
r_conic = p_def/(1 + e*sp.cos(theta))

# Steps 23-25: Ellipse parameters and Kepler's 1st law
# For ellipse: a = p/(1-e²), so p = a*(1-e²)
# Final form: r(theta) = a*(1-e²)/(1 + e*cos(theta))
p_ellipse = a*(1 - e**2)
r_kepler1 = (a*(1-e**2))/(1 + e*sp.cos(theta))

# Verify consistency: p_def = p_ellipse
# This gives us h² = GM*a*(1-e²)

# Step 26: Relationship h² = GM*a*(1-e²)
h_squared_orbital = G*M*a*(1 - e**2)
h_from_orbital = sp.sqrt(h_squared_orbital)

# Step 27: Ellipse area A = π*a*b where b = a*sqrt(1-e²)
b_def = a*sp.sqrt(1 - e**2)
A_ellipse = pi*a*b_def
A_expanded = pi*a**2*sp.sqrt(1 - e**2)
assert sp.simplify(A_ellipse - A_expanded) == 0

# Step 28: Period from T = A_ellipse / dot(A)
# Using dot(A) = h/2 from Kepler's 2nd law
T_from_area = A_ellipse / (h/2)
T_step28 = (2*pi*a**2*sp.sqrt(1 - e**2)) / h
assert sp.simplify(T_from_area - T_step28) == 0

# Step 29: Already simplified in step 28

# Step 30: Substitute h = sqrt(GM*a*(1-e²))
T_step30 = T_step28.subs(h, h_from_orbital)
T_substituted = (2*pi*a**2*sp.sqrt(1 - e**2)) / sp.sqrt(G*M*a*(1 - e**2))
assert sp.simplify(T_step30 - T_substituted) == 0

# Step 31: Cancel sqrt(1-e²) terms
T_step31 = (2*pi*a**(sp.Rational(3,2))) / sp.sqrt(G*M)
T_simplified = sp.simplify(T_substituted)
assert sp.simplify(T_simplified - T_step31) == 0

# Step 32: Square to get Kepler's 3rd law
T_squared = T_step31**2
T_squared_kepler3 = (4*pi**2*a**3)/(G*M)
assert sp.simplify(T_squared - T_squared_kepler3) == 0

# Verify special cases of Kepler's 1st law
# At periapsis (θ = 0): r = a(1-e)
r_periapsis = r_kepler1.subs(theta, 0)
expected_periapsis = a*(1-e)
assert sp.simplify(r_periapsis - expected_periapsis) == 0

# At apoapsis (θ = π): r = a(1+e)
r_apoapsis = r_kepler1.subs(theta, pi)
expected_apoapsis = a*(1+e)
assert sp.simplify(r_apoapsis - expected_apoapsis) == 0

# Circular orbit limit (e = 0): r = a
r_circular = r_kepler1.subs(e, 0)
assert sp.simplify(r_circular - a) == 0

# Verify Kepler's 3rd law with solar system data
# Earth: a ≈ 1 AU, T ≈ 1 year
# Mars: a ≈ 1.52 AU, T ≈ 1.88 years
a_earth, a_mars = 1, sp.Rational(152, 100)
T_earth, T_mars = 1, sp.Rational(188, 100)

# Test T₁²/T₂² = a₁³/a₂³
ratio_periods = T_earth**2 / T_mars**2
ratio_distances = a_earth**3 / a_mars**3
difference = abs(float(ratio_periods - ratio_distances))
assert difference < 0.01  # Within 1% accuracy

print('All Kepler laws verified successfully!')
print(f'Kepler 1st law: r(θ) = a(1-e²)/(1+e cos θ)')
print(f'Kepler 2nd law: dA/dt = L/(2m) = h/2 = constant')
print(f'Kepler 3rd law: T² = 4π²a³/(GM)')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=keplers_laws.json](https://theoria-dataset.org/entries.html?entry=keplers_laws.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0